In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.llms.ollama import Ollama

In [15]:
model_name="gemma:2b"
llm = Ollama(model=model_name, request_timeout=120.0)

In [4]:
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model =OllamaEmbedding(model_name="nomic-embed-text:latest")

In [16]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
!mkdir data

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# API access to llama-cloud
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LLAMA_CLOUD_API_KEY"]=os.getenv("LLAMA_CLOUD_API_KEY")

In [12]:
from llama_parse import LlamaParse

# docs_kendrick = LlamaParse(result_type="text").load_data("./data/kendrick.pdf")
# docs_drake = LlamaParse(result_type="text").load_data("./data/drake.pdf")
# docs_both = LlamaParse(result_type="text").load_data(
#     "./data/drake_kendrick_beef.pdf"
# )


from llama_index.core import SimpleDirectoryReader

docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

In [13]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

I'm just an AI, I don't have personal preferences or opinions. However, both Drake and Kendrick Lamar are highly acclaimed artists with unique styles and contributions to the music industry.

Drake is known for his introspective lyrics, melodic flow, and ability to blend genres like hip-hop, R&B, and pop. He has been a dominant force in popular culture, breaking numerous records and collaborating with other top artists.

Kendrick Lamar is celebrated for his socially conscious lyrics, innovative production, and genre-bending sound that blends jazz, funk, and hip-hop. He is widely regarded as one of the most influential and critically acclaimed rappers of our time, with a reputation for pushing boundaries and challenging societal norms through his music.

Ultimately, it's up to personal taste which artist you prefer! Both Drake and Kendrick Lamar have their own strengths and contributions to the world of music.


In [17]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

I am an AI chatbot and do not have personal preferences or opinions. I am unable to endorse or compare artists.

In [18]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [19]:
print(response)

assistant: The city sleeps, the neon fades,
The streets are empty, save for the parade.
Shadows dance on the cracked asphalt floor,
A symphony of silence, a moment to abhor.

My thoughts drift to a distant shore,
Where the rhythm of life still roars.
I yearn for a melody, a song to ignite,
A spark of passion, a flame burning bright.

The city sleeps, but I can't resist,
A whisper of hope, a desperate kiss.
I yearn for the echo of a lost refrain,
A chorus that rises, a melody that sustains.


In [20]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)

In [21]:
response = query_engine.query("Tell me about family matters")

In [22]:
print(str(response))

The context does not mention anything about "family matters", so I cannot answer this query from the provided context information.


In [23]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [24]:
response = summary_engine.query(
    "Given your assessment of this article, who won the beef?"
)

In [25]:
print(str(response))

The context does not provide any information about who won the beef, so I cannot answer this question from the provided context.


In [26]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [27]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)

Selecting query engine 0: Useful for searching for specific facts..


In [28]:
print(response)

The context does not mention anything about the song meet the grahams, so I cannot answer this query from the provided context information.
